## Tobigs 5주차 Crawling 과제
- Selenium을 활용하여 학술연구정보서비스 RISS에서 주어진 키워드에 대한 **학위 논문**을 검색 후 **첫번째 페이지**에 존재하는 논문의 **URL**을 먼저 수집합니다. 
- 수집한 URL을 기반으로 접근하여 논문 정보 **1) 저자 2) 발행년도 3) 제목 4) 부가정보의 첫번째 항목**을 수집해주세요! 
- 주석을 읽고 주어진 코드 중간에 있는 ???와 공란을 채워주시면 됩니다!

In [2]:
! pip install selenium

  Using cached selenium-3.141.0-py2.py3-none-any.whl (904 kB)

You should consider upgrading via the 'c:\ana3\envs\t_f2.2\python.exe -m pip install --upgrade pip' command.


In [18]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd

In [19]:
keyword='코로나'  #검색용 키워드
paper=[]  #결과 저장용 리스트

#### 1. 드라이버 오픈 및 Riss로 이동

In [20]:
driver = webdriver.Chrome('C:/Users/김건우/Downloads/chromedriver/chromedriver.exe')  #드라이버 오픈
driver.get('http://www.riss.kr/index.do')   #riss로 이동

#### 2. 검색창에 접근하여 검색용 키워드 입력 및 검색

In [21]:
driver.find_element_by_id('query').send_keys(keyword)  # 검색창 접근 및 키워드 입력
driver.find_element_by_id('query').submit()   # 검색창에 검색용 키워드 검색

#### 3. 검색 화면 창에서 학위 논문 창으로 이동
![image](https://user-images.githubusercontent.com/69336270/129825390-d270bcca-003c-46f4-8bfd-14e4f1ed3f87.png)
  
    방법 1. 상단의 학위 논문 탭 클릭  
    방법 2. 검색 화면 창에서 학위논문 결과 옆 더보기 버튼 클릭

In [25]:
driver.find_element_by_xpath('//*[@id="tabMenu"]/div/ul/li[3]/a').send_keys(Keys.ENTER)

#### 4. 주어진 키워드로 검색했을 때 학위 논문의 전체 수 구하기

![image](https://user-images.githubusercontent.com/69336270/129825100-89dd9681-4dd7-451a-804f-e0fbe437aeba.png)

In [27]:
total=driver.find_element_by_xpath('//*[@id="divContent"]/div[2]/div/div[1]/dl/dd/span/span').text  #전체 논문수
print(total)
""" xpath를 통해 구한 전체 논문수 total을 특문 제거 후 int형으로 변환 """
total = int(total.replace(',',''))
print(total, type(total))

2,296
2296 <class 'int'>


#### 5. 첫페이지에 존재하는 논문의 상세정보 url 수집

In [28]:
urls = []

html = driver.page_source # 페이지 소스 저장
""" html 소스코드를 BeautifulSoup을 이용해 파싱 """

if total < 10:
    # 주어진 키워드로 검색 시 전체 논문 수가 10개 미만인 경우 - 논문 첫 페이지의 논문 수 = 전체 논문 수
    for i in range(1, total+1):  
        page = '/html/body/div[1]/div[2]/div[4]/div[2]/div/div[3]/div[2]/ul/li['+str(i)+']/div[2]/p[1]/a'
        url=driver.find_element_by_xpath(page).get_attribute('href')
        urls.append(url)
        print(url)
else:
    # 주어진 키워드로 검색 시 전체 논문 수가 10개 이상인 경우 - 논문 첫 페이지의 논문 수 = 10개
    for i in range(1, 11):
        page = '/html/body/div[1]/div[2]/div[4]/div[2]/div/div[3]/div[2]/ul/li['+str(i)+']/div[2]/p[1]/a'
        url=driver.find_element_by_xpath(page).get_attribute('href')
        urls.append(url)
        print(url)

http://www.riss.kr/search/detail/DetailView.do?p_mat_type=be54d9b8bc7cdb09&control_no=a54bb7f34c8453bfffe0bdc3ef48d419
http://www.riss.kr/search/detail/DetailView.do?p_mat_type=be54d9b8bc7cdb09&control_no=3b8639661acf0be8ffe0bdc3ef48d419
http://www.riss.kr/search/detail/DetailView.do?p_mat_type=be54d9b8bc7cdb09&control_no=29ccc72d95b8f0dbffe0bdc3ef48d419
http://www.riss.kr/search/detail/DetailView.do?p_mat_type=be54d9b8bc7cdb09&control_no=b6e00fd98cafeb39ffe0bdc3ef48d419
http://www.riss.kr/search/detail/DetailView.do?p_mat_type=be54d9b8bc7cdb09&control_no=8b2e61b36976fa93ffe0bdc3ef48d419
http://www.riss.kr/search/detail/DetailView.do?p_mat_type=be54d9b8bc7cdb09&control_no=dec269f74836aea5ffe0bdc3ef48d419
http://www.riss.kr/search/detail/DetailView.do?p_mat_type=be54d9b8bc7cdb09&control_no=2f61e1173312d7e6ffe0bdc3ef48d419
http://www.riss.kr/search/detail/DetailView.do?p_mat_type=be54d9b8bc7cdb09&control_no=6ea66a8b57ca813effe0bdc3ef48d419
http://www.riss.kr/search/detail/DetailView.do?p

#### 6-1. 저자 정보 구하기 예제
수집한 url 중 첫번째 링크의 저자 정보를 구해보자!

In [29]:
data = []  # 수집한 정보 저장용 리스트
driver.get(urls[0])   # 첫번째 링크로 이동

In [31]:
""" 저자 정보를 xpath와 .text 메소드를 통해 구해주세요! """
author = driver.find_element_by_xpath('//*[@id="thesisInfoDiv"]/div[2]/div[1]/ul/li[1]/div/p/a').text
data.append(author)
print(author)

맹영연


#### 6-2. 연도 정보 구하기 예제
수집한 url 중 첫번째 링크의 연도 정보를 구해보자!

In [32]:
""" 연도 정보를 xpath와 .text 메소드를 통해 구해주세요! """
year = driver.find_element_by_xpath('//*[@id="thesisInfoDiv"]/div[2]/div[1]/ul/li[4]/div/p').text
data.append(year)
print(year)

2021


#### 6-3. 제목 정보 구하기 예제
수집한 url 중 첫번째 링크의 제목 정보를 구해보자!

In [33]:
""" 제목 정보를 xpath와 .text 메소드를 통해 구해주세요! """
title = driver.find_element_by_xpath('//*[@id="thesisInfoDiv"]/div[1]/h3').text
data.append(title)
print(title)

코로나19에 대한 계획행동 요인들이 관련 뉴스 공유의도 및 대처 행위의도에 미치는 영향에 관한 연구 = Exploring the Influence of Planned Behavioral Factors on COVID-19 News Sharing and Coping Behavioral Intention


#### 6-4. 부가정보 정보 구하기 예제
수집한 url 중 첫번째 링크의 부가정보 정보를 구해보자!

![image](https://user-images.githubusercontent.com/69336270/129768125-b13b9554-7dba-44e0-aa27-07b901a7f50c.png)
  
부가정보에서 데이터를 수집하기 위해서는 우선 더보기 버튼을 클릭해주어야 한다. 이때, 더보기 버튼의 **클래스명**을 통해 클릭해주자!

In [34]:
# 부가정보 가져오기 - 더보기 버튼 클릭

moreViews = driver.find_elements_by_class_name('moreView')

for moreView in moreViews:
    moreView.send_keys(Keys.ENTER)
    sleep(0.1)

In [36]:
# 부가정보 가져오기 - 첫번째 항목만 가져오기

try:
    """ 부가정보의 첫번째 항목을 xpath와 .text 메소드를 통해 구해주세요! """
    first_info = driver.find_element_by_xpath('//*[@id="additionalInfoDiv"]/div/div[1]/div[3]/p').text

    
    data.append(first_info)
    print(first_info)
except:
    data.append("부가정보가 없습니다.")

When COVID-19 happened, people were increasingly demanding related information. Sufficient awareness of infectious disease-related crises and judgment on the possibility of occurrence are needed. The most important knowledge is how to effectively protect one's safety, which has a significant impact on people taking precautions. It is also very difficult to prevent various emergencies in advance because there is uncertainty about the occurrence of them. If news and information resources of various emergencies can be shared with the media to discover, report, and take real-time action, it will improve the government and
individuals' urgent response and reduce losses. However, in practice, after inmates obtain Covid-19 information from the news, they often share it to other inmates without confirming the truth. Especially in the
Internet age, news is actively distributed and consumed around online media, so inmates deliver news information that people around them like or need through onli

#### 6-5. 수집한 정보를 바탕으로 DataFrame 만들기

In [37]:
pd.DataFrame(data = {'저자':data[0], '발행년도':data[1], '제목':data[2], '부가정보':data[3], 'Url':urls[0]}, index = [0])

,저자,발행년도,제목,부가정보,Url
0,맹영연,2021,코로나19에 대한 계획행동 요인들이 관련 뉴스 공유의도 및 대처 행위의도에 미치는 ...,부가정보가 없습니다.,http://www.riss.kr/search/detail/DetailView.do...


#### 7. 논문 정보 수집 과정 함수화 하기
앞선 예제를 기반으로 함수 속 ???와 공란을 채워주세요

In [38]:
def store(url):
    data=[]  #빈 리스트
    authur=""  #저자
    year=""  #연도
    title=""  #제목
    first_info=""  #부가정보
    
    driver.get(url)   # 첫번째 링크로 이동
    
    #저자
    """ 저자 정보의 첫번째 항목을 xpath와 .text 메소드를 통해 구해주세요! """
    author = driver.find_element_by_xpath('//*[@id="thesisInfoDiv"]/div[2]/div[1]/ul/li[1]/div/p/a').text

    #연도
    """ 연도 정보의 첫번째 항목을 xpath와 .text 메소드를 통해 구해주세요! """
    year = driver.find_element_by_xpath('//*[@id="thesisInfoDiv"]/div[2]/div[1]/ul/li[4]/div/p').text

    #제목
    """ 제목 정보의 첫번째 항목을 xpath와 .text 메소드를 통해 구해주세요! """
    title = driver.find_element_by_xpath('//*[@id="thesisInfoDiv"]/div[1]/h3').text

    #부가정보    
    moreViews = driver.find_elements_by_class_name('moreView')

    for moreView in moreViews:
        moreView.send_keys(Keys.ENTER)
        sleep(0.1)

    try:
        """ 부가정보의 첫번째 항목을 xpath와 .text 메소드를 통해 구해주세요! """
        first_info = driver.find_element_by_xpath('//*[@id="additionalInfoDiv"]/div/div[1]/div[3]/p').text

    except:
        first_info = "부가정보가 없습니다."
        
    data.append(author)
    data.append(year)
    data.append(title)
    data.append(first_info)
    data.append(url)
    paper.append(data)

#### 8. 수집 및 저장
해당 코드는 채점을 위한 코드로 수정하시면 안됩니다!

In [39]:
paper=[]
for url in urls:
    driver.get(url)
    store(url)
    sleep(1)

driver.close()  # 드라이버 닫기

paper_info = pd.DataFrame(paper, columns=['저자', '발행년도', '제목', '부가정보', 'Url'])
paper_info.to_csv('./paper_info.csv', encoding='utf-8')
paper_info

,저자,발행년도,제목,부가정보,Url
0,맹영연,2021,코로나19에 대한 계획행동 요인들이 관련 뉴스 공유의도 및 대처 행위의도에 미치는 ...,"When COVID-19 happened, people were increasing...",http://www.riss.kr/search/detail/DetailView.do...
1,이효원,2021,포스트 코로나 시대의 직선형 공원 공공 브랜드 디자인 제안 - 경의선 숲길 공원을 ...,본 연구는 포스트코로나시대의 직선형공원 공공 브랜드 디자인 제안: 경의선숲길을 중심...,http://www.riss.kr/search/detail/DetailView.do...
2,정용철,2008,코로나 방전형 정전기 제거장치에서의 이온생성 최적화 조건에 관한 연구 = A Stu...,The purpose of this study is to analyze the op...,http://www.riss.kr/search/detail/DetailView.do...
3,홍은영,2021,원장의 코로나19 관리리더십이 영유아교사의 조직몰입과 직무스트레스에 미치는 영향: ...,본 연구는 전국에 영유아교육기관에 근무하는 영유아교사를 대상으로 하여 원장의 코로나...,http://www.riss.kr/search/detail/DetailView.do...
4,박석훈,2014,코로나방식 필터대전 조건에 따른 공기정화필터의 여과특성 = Filtration Ch...,"쾌적한 실내공간, 초정밀 제품 생산, 개인위생보호 등의 여러 목적으로 공기정화필터의...",http://www.riss.kr/search/detail/DetailView.do...
5,김시영,2021,코로나19 상황에서 유아교사의 적응과정과 극복요인 = The Adaptation P...,2019년 12월 코로나19의 발발은 불과 4개월 만에 전 세계를 팬데믹 상황으로 ...,http://www.riss.kr/search/detail/DetailView.do...
6,배민영,2021,코로나-19 스트레스가 보육교사의 삶의 질에 미치는 영향 : 긍정심리자본과 회복탄력...,"본 연구는 서울, 경기지역에 소재한 어린이집에 종사하는 보육교사를 대상으로 하여 코...",http://www.riss.kr/search/detail/DetailView.do...
7,박지원,2021,코로나19 상황에서의 미디어 이용과 미디어 신뢰가 코로나19 감염 가능성에 대한 낙...,부가정보가 없습니다.,http://www.riss.kr/search/detail/DetailView.do...
8,"Zhang, Yuqing",2021,코로나19 예방행위 지속의도에 영향을 미치는 요인 연구 : 건강신념모델(HBM)의 적용,사람들이 질병 혹은 위험의 특성을 지각하는 것에 따라 우리는 그들의 예방 행동을 예...,http://www.riss.kr/search/detail/DetailView.do...
9,강찬희,2021,동적토픽모델링과 의미연결망을 통한 코로나19 이슈분석 : 트위터 텍스트데이터를 활용하여,중국 우한을 필두로 시작된 신종코로나 바이러스는 코로나19(COVID-19)로 명명...,http://www.riss.kr/search/detail/DetailView.do...
